In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 불러오기

df = pd.read_csv('../processed_data/processed_features.csv')

print(f"데이터 크기: {df.shape}")
print(f"컬럼: {list(df.columns)}")
print("\n첫 5행:")
print(df.head())

데이터 크기: (2156, 36)
컬럼: ['file', 'ch1_rms', 'ch1_peak', 'ch1_std', 'ch1_kurtosis', 'ch2_rms', 'ch2_peak', 'ch2_std', 'ch2_kurtosis', 'ch3_rms', 'ch3_peak', 'ch3_std', 'ch3_kurtosis', 'ch4_rms', 'ch4_peak', 'ch4_std', 'ch4_kurtosis', 'ch5_rms', 'ch5_peak', 'ch5_std', 'ch5_kurtosis', 'ch6_rms', 'ch6_peak', 'ch6_std', 'ch6_kurtosis', 'ch7_rms', 'ch7_peak', 'ch7_std', 'ch7_kurtosis', 'ch8_rms', 'ch8_peak', 'ch8_std', 'ch8_kurtosis', 'timestamp', 'time_to_failure', 'days_to_failure']

첫 5행:
                  file   ch1_rms  ch1_peak   ch1_std  ch1_kurtosis   ch2_rms  \
0  2003.10.22.12.06.24  0.124614     0.720  0.081122      1.069717  0.117493   
1  2003.10.22.12.09.13  0.123811     0.654  0.079515      1.162128  0.116833   
2  2003.10.22.12.14.13  0.125246     0.623  0.080217      0.986819  0.118384   
3  2003.10.22.12.19.13  0.125197     0.598  0.080825      1.034839  0.119005   
4  2003.10.22.12.24.13  0.125618     0.623  0.082034      1.110728  0.119688   

   ch2_peak   ch2_std  ch2_ku

In [28]:
# 2. 베어링별 개별 데이터셋 생성

# 고장 임계값 설정 (기존 데이터상 5.3, 6.3일부터 고장 데이터를 확인하였으므로 6일로 설정)
threshold_days = 6

# 베어링별 채널 매핑 및 고장 여부
bearing_config = {
    1: {'channels': ['ch1_rms', 'ch1_peak', 'ch1_std', 'ch1_kurtosis', 
                    'ch2_rms', 'ch2_peak', 'ch2_std', 'ch2_kurtosis'], 'faulty': False},
    2: {'channels': ['ch3_rms', 'ch3_peak', 'ch3_std', 'ch3_kurtosis',
                    'ch4_rms', 'ch4_peak', 'ch4_std', 'ch4_kurtosis'], 'faulty': False},
    3: {'channels': ['ch5_rms', 'ch5_peak', 'ch5_std', 'ch5_kurtosis',
                    'ch6_rms', 'ch6_peak', 'ch6_std', 'ch6_kurtosis'], 'faulty': True},
    4: {'channels': ['ch7_rms', 'ch7_peak', 'ch7_std', 'ch7_kurtosis',
                    'ch8_rms', 'ch8_peak', 'ch8_std', 'ch8_kurtosis'], 'faulty': True}
}

# 통합 데이터셋 생성
unified_data = []

for bearing_num, config in bearing_config.items():
    # 베어링별 데이터 추출
    bearing_data = df[['timestamp', 'days_to_failure'] + config['channels']].copy()
    
    # 베어링별 라벨링
    if config['faulty']:
        bearing_data['label'] = (bearing_data['days_to_failure'] <= threshold_days).astype(int)
    else:
        bearing_data['label'] = 0
    
    # 특성 컬럼명 표준화
    feature_mapping = {ch: f'feature_{i+1}' for i, ch in enumerate(config['channels'])}
    bearing_data = bearing_data.rename(columns=feature_mapping)
    
    # 베어링 식별자 추가
    bearing_data['bearing_id'] = bearing_num
    
    unified_data.append(bearing_data)

# 통합 데이터프레임 생성
unified_df = pd.concat(unified_data, ignore_index=True)

print(f"통합 데이터셋: {unified_df.shape}")
print(f"베어링별 라벨 분포:")
for bearing_num in [1, 2, 3, 4]:
    bearing_subset = unified_df[unified_df['bearing_id'] == bearing_num]
    failure_rate = bearing_subset['label'].mean() * 100
    print(f"  Bearing {bearing_num}: {failure_rate:.1f}% 고장률")

# 특성 컬럼 정의
feature_cols = [col for col in unified_df.columns if col.startswith('feature_')]
print(f"특성 수: {len(feature_cols)}개")

통합 데이터셋: (8624, 12)
베어링별 라벨 분포:
  Bearing 1: 0.0% 고장률
  Bearing 2: 0.0% 고장률
  Bearing 3: 30.1% 고장률
  Bearing 4: 30.1% 고장률
특성 수: 8개


In [29]:
# 3. 데이터 분할

from sklearn.model_selection import train_test_split

# 계층적 분할 (라벨 비율 유지)
train_data, temp_data = train_test_split(
    unified_df, 
    test_size=0.3,  # 30%를 val+test로 
    stratify=unified_df['label'],  # 라벨 비율 유지
    random_state=42
)

val_data, test_data = train_test_split(
    temp_data,
    test_size=0.5,  # temp의 50%씩을 val, test로
    stratify=temp_data['label'],  # 라벨 비율 유지  
    random_state=42
)

print(f"계층적 분할 결과:")
print(f"  Train: {len(train_data):,}개")
print(f"  Val: {len(val_data):,}개") 
print(f"  Test: {len(test_data):,}개")

# 라벨 분포 확인
for name, data in [("Train", train_data), ("Val", val_data), ("Test", test_data)]:
    total = len(data)
    failure_count = data['label'].sum()
    failure_rate = data['label'].mean() * 100
    normal_count = total - failure_count
    print(f"{name}: 정상 {normal_count:4d}개 ({100-failure_rate:5.1f}%), 고장 {failure_count:3d}개 ({failure_rate:5.1f}%)")

# 베어링별 분포 확인
print(f"\n베어링별 분할 확인:")
for bearing_num in [1, 2, 3, 4]:
    print(f"Bearing {bearing_num}:")
    for name, data in [("Train", train_data), ("Val", val_data), ("Test", test_data)]:
        bearing_subset = data[data['bearing_id'] == bearing_num]
        total = len(bearing_subset)
        failure_count = bearing_subset['label'].sum()
        failure_rate = bearing_subset['label'].mean() * 100
        print(f"  {name}: {total:4d}개 ({failure_rate:5.1f}% 고장률)")
    print()

계층적 분할 결과:
  Train: 6,036개
  Val: 1,294개
  Test: 1,294개
Train: 정상 5128개 ( 85.0%), 고장 908개 ( 15.0%)
Val: 정상 1099개 ( 84.9%), 고장 195개 ( 15.1%)
Test: 정상 1099개 ( 84.9%), 고장 195개 ( 15.1%)

베어링별 분할 확인:
Bearing 1:
  Train: 1505개 (  0.0% 고장률)
  Val:  323개 (  0.0% 고장률)
  Test:  328개 (  0.0% 고장률)

Bearing 2:
  Train: 1497개 (  0.0% 고장률)
  Val:  324개 (  0.0% 고장률)
  Test:  335개 (  0.0% 고장률)

Bearing 3:
  Train: 1511개 ( 29.3% 고장률)
  Val:  336개 ( 32.7% 고장률)
  Test:  309개 ( 31.4% 고장률)

Bearing 4:
  Train: 1523개 ( 30.6% 고장률)
  Val:  311개 ( 27.3% 고장률)
  Test:  322개 ( 30.4% 고장률)



In [26]:
# 4. 시계열 윈도우 생성

window_size = 10

def create_sequences(data, feature_cols, window_size):
    X, y = [], []
    
    # 베어링별로 시퀀스 생성
    for bearing_num in [1, 2, 3, 4]:
        bearing_subset = data[data['bearing_id'] == bearing_num].sort_values('timestamp')
        
        for i in range(len(bearing_subset) - window_size):
            window_data = bearing_subset.iloc[i:i+window_size][feature_cols].values
            target_label = bearing_subset.iloc[i+window_size]['label']
            
            X.append(window_data)
            y.append(target_label)
    
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, feature_cols, window_size)
X_val, y_val = create_sequences(val_data, feature_cols, window_size)
X_test, y_test = create_sequences(test_data, feature_cols, window_size)

print(f"시계열 윈도우 생성 완료:")
print(f"  X_train: {X_train.shape} (samples, timesteps, features)")
print(f"  X_val: {X_val.shape}")
print(f"  X_test: {X_test.shape}")

# 윈도우 생성 후 라벨 분포
print(f"\n윈도우 생성 후 라벨 분포:")
for name, y in [("y_train", y_train), ("y_val", y_val), ("y_test", y_test)]:
    total = len(y)
    failure_count = y.sum()
    failure_rate = y.mean() * 100
    normal_count = total - failure_count
    print(f"{name}: 정상 {normal_count:4d}개 ({100-failure_rate:5.1f}%), 고장 {failure_count:3d}개 ({failure_rate:5.1f}%)")

시계열 윈도우 생성 완료:
  X_train: (5996, 10, 8) (samples, timesteps, features)
  X_val: (1254, 10, 8)
  X_test: (1254, 10, 8)

윈도우 생성 후 라벨 분포:
y_train: 정상 5088개 ( 84.9%), 고장 908개 ( 15.1%)
y_val: 정상 1059개 ( 84.4%), 고장 195개 ( 15.6%)
y_test: 정상 1059개 ( 84.4%), 고장 195개 ( 15.6%)


In [31]:
# 5. 데이터 정규화

# 3D 데이터를 2D로 변환하여 정규화
X_train_2d = X_train.reshape(-1, X_train.shape[-1])
X_val_2d = X_val.reshape(-1, X_val.shape[-1])
X_test_2d = X_test.reshape(-1, X_test.shape[-1])

# 정규화 적용
scaler = StandardScaler()
scaler.fit(X_train_2d)

X_train_scaled_2d = scaler.transform(X_train_2d)
X_val_scaled_2d = scaler.transform(X_val_2d)
X_test_scaled_2d = scaler.transform(X_test_2d)

# 3D 형태로 복원
X_train_scaled = X_train_scaled_2d.reshape(X_train.shape)
X_val_scaled = X_val_scaled_2d.reshape(X_val.shape)
X_test_scaled = X_test_scaled_2d.reshape(X_test.shape)

print(f"정규화 완료:")
print(f"  스케일러 특성 수: {scaler.n_features_in_}")
print(f"  정규화 후 평균: {X_train_scaled_2d.mean():.6f}")
print(f"  정규화 후 표준편차: {X_train_scaled_2d.std():.6f}")

정규화 완료:
  스케일러 특성 수: 8
  정규화 후 평균: 0.000000
  정규화 후 표준편차: 1.000000


In [32]:
# 6. 전처리된 데이터 저장

processed_data_dir = '../processed_data'
models_dir = '../models'

os.makedirs(processed_data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# 데이터 저장
joblib.dump(X_train_scaled, f'{processed_data_dir}/X_train_scaled.pkl')
joblib.dump(X_val_scaled, f'{processed_data_dir}/X_val_scaled.pkl')
joblib.dump(X_test_scaled, f'{processed_data_dir}/X_test_scaled.pkl')

joblib.dump(y_train, f'{processed_data_dir}/y_train.pkl')
joblib.dump(y_val, f'{processed_data_dir}/y_val.pkl')
joblib.dump(y_test, f'{processed_data_dir}/y_test.pkl')

joblib.dump(scaler, f'{models_dir}/scaler.pkl')

print(f"데이터 저장 완료:")
print(f"  X_train_scaled.pkl: {X_train_scaled.shape}")
print(f"  X_val_scaled.pkl: {X_val_scaled.shape}")
print(f"  X_test_scaled.pkl: {X_test_scaled.shape}")
print(f"  scaler.pkl: 정규화 스케일러")

데이터 저장 완료:
  X_train_scaled.pkl: (5996, 10, 8)
  X_val_scaled.pkl: (1254, 10, 8)
  X_test_scaled.pkl: (1254, 10, 8)
  scaler.pkl: 정규화 스케일러
